## 从简单任务开始

In [1]:
import openai
import os

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

#COMPLETION_MODEL = "GPT35"
COMPLETION_MODEL = "text-davinci-003"


def get_response(prompt):
    completions = openai.Completion.create (
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.0,        
    )
    message = completions['choices'][0].text # type: ignore
    return message

幻觉-胡说

In [2]:
# 基本上会胡扯
print(get_response("Imagento算法是什么？"))



Imagento算法是一种基于深度学习的图像分类算法，它可以自动识别图像中的物体，并将其分类为不同的类别。它可以用于图像分类、图像检索、图像识别等应用。


### 试一下命名实体识别

In [3]:
sentence = "Man Utd must win trophies, says Ten Hag ahead of League Cup final"
prompt = f"请提取以下句子中的命名实体，并以json形式输出\n{sentence} \n"

print(get_response(prompt))


{
    "实体": ["Man Utd", "Ten Hag", "League Cup"]
}


In [4]:
prompt = f"请提取以下句子中的命名实体，并以json形式输出，json的key是实体的类型。\n{sentence} \n"
print(get_response(prompt))


{
    "Organization": ["Man Utd", "League Cup"],
    "Person": ["Ten Hag"]
}


In [5]:
order_sentence = "你好，我有一个订单一直没有收到，订单号是202303251200ABC。"
prompt = f"请提取以下句子中的命名实体，并以json形式输出，json的key是实体的类型。\n{order_sentence} \n"
print(get_response(prompt))


{
    "Order": "202303251200ABC"
}


### 情感分析

In [6]:
prompts = """判断一下以下用户的评论情感上是正面的还是负面的.

买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质。"""

print(get_response(prompts))



正面的。


### 更好的Prompt来控制结果输出

In [7]:
prompts = """判断一下用户的评论情感上是正面的还是负面的.

评论：买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质。
情感："""

print(get_response(prompts))

正面


## Few-shot Learning

In [8]:
prompts = """判断一下用户的评论情感上是正面的还是负面的
评论：买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质
情感：正面

评论：随意降价，不予价保，服务态度差
情感：负面
"""

good_case = prompts + """
评论：外形外观：苹果审美一直很好，金色非常漂亮
拍照效果：14pro升级的4800万像素真的是没的说，太好了，
运行速度：苹果的反应速度好，用上三五年也不会卡顿的，之前的7P用到现在也不卡
其他特色：14pro的磨砂金真的太好看了，不太高调，也不至于没有特点，非常耐看，很好的
情感：
"""

print(get_response(good_case))

正面


### 对于Few-Shot Learning的封装

In [ ]:
class Example:
    def __init__(self, question, answer) -> None:
        self.question = question
        self.answer = answer

class PromptTemplate:
    def __init__(self, prompt, examples, question_prefix = "Q: ", answer_prefix = "A: ") -> None:
        self.prompt = prompt
        self.examples = examples
        self.question_prefix = question_prefix
        self.answer_prefix = answer_prefix

    def get_prompt(self, question):
        result = self.prompt + "\n\n"
        for example in self.examples:
            result += f"{self.question_prefix} {example.question}\n{self.answer_prefix} {example.answer}\n\n"
        result += f"{self.question_prefix} {question}\n{self.answer_prefix} "
        return result

example1 = Example("买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质", "正面")
example2 = Example("随意降价，不予价保，服务态度差", "负面")

prompt = PromptTemplate("判断一下用户的评论情感上是正面的还是负面的", [example1, example2], "评论：", "情感：")

question = """外形外观：苹果审美一直很好，金色非常漂亮
拍照效果：14pro升级的4800万像素真的是没的说，太好了，
运行速度：苹果的反应速度好，用上三五年也不会卡顿的，之前的7P用到现在也不卡
其他特色：14pro的磨砂金真的太好看了，不太高调，也不至于没有特点，非常耐看，很好的"""

print(f"Prompt:")
print(prompt.get_prompt(question))
print("\n\n")
print(f"Response:")
print(get_response(prompt.get_prompt(question)))




## Completions接口

### max_tokens, temperature, top_p, n 以及 stop 参数的演示

In [9]:
import openai
import os

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

#COMPLETION_MODEL = "GPT35"
COMPLETION_MODEL = "text-davinci-003"

def get_response(prompt, max_tokens=512, temperature=0.0, n=1, stop=None):
    completions = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=max_tokens,
        n=n,
        stop=stop,
        temperature=temperature,        
    )
    return completions


In [10]:
prompt = """请你用淘宝客服的语气告诉用户，这个洗面奶的优惠已经结束了。"""

answer = get_response(prompt)
print(answer)

{
  "id": "cmpl-7hBdFl6drlk9w96w0d1YZCz8wffLu",
  "object": "text_completion",
  "created": 1690527173,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n\u60a8\u597d\uff0c\u975e\u5e38\u62b1\u6b49\u7684\u901a\u77e5\u60a8\uff0c\u8fd9\u6b3e\u6d17\u9762\u5976\u7684\u4f18\u60e0\u6d3b\u52a8\u5df2\u7ecf\u7ed3\u675f\uff0c\u611f\u8c22\u60a8\u7684\u5173\u6ce8\uff0c\u671f\u5f85\u60a8\u7684\u4e0b\u6b21\u5149\u4e34\uff01",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 102,
    "prompt_tokens": 61,
    "total_tokens": 163
  }
}


In [11]:
print(answer.choices[0].text) # type: ignore



您好，非常抱歉的通知您，这款洗面奶的优惠活动已经结束，感谢您的关注，期待您的下次光临！


#### max_token限制

In [12]:
answer = get_response(prompt, max_tokens=10)
print(answer)
print(answer.choices[0].text) # type: ignore

{
  "id": "cmpl-7hBeCAQoD2LwxxC2ahmDq065PXDfa",
  "object": "text_completion",
  "created": 1690527232,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n\u60a8\u597d\uff0c",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 61,
    "total_tokens": 71
  }
}


您好，


#### 同时返回 多个 回答 ；并且通过温度，调节回答内容发散度

In [13]:
answer = get_response(prompt, temperature=1, n=3)
print(answer)
print("Completions:")
for choice in answer.choices: # type: ignore
    print(choice.text) # type: ignore
    print("\n--\n")


{
  "id": "cmpl-7hBf50HjvcVmNNUDeecmouSzURddY",
  "object": "text_completion",
  "created": 1690527287,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\n\u5c0a\u656c\u7684\u7528\u6237\uff1a\n\u60a8\u597d\uff01\u5f88\u62b1\u6b49\u7ed9\u60a8\u5e26\u6765\u4e0d\u4fbf\u3002\u5f88\u9057\u61be\u7684\u901a\u77e5\u60a8\uff0c\u8fd9\u4e2a\u6d17\u9762\u5976\u7684\u4f18\u60e0\u6d3b\u52a8\u5df2\u7ecf\u7ed3\u675f\u4e86\uff0c\u671f\u5f85\u60a8\u4e0b\u6b21\u6709\u66f4\u591a\u7684\u4f18\u60e0\u53ef\u4ee5\u53c2\u4e0e\u54e6\u3002\u671f\u95f4\u5982\u679c\u8fd8\u6709\u4ec0\u4e48\u95ee\u9898\uff0c\u90fd\u53ef\u4ee5\u968f\u65f6\u8054\u7cfb\u6211\u4eec\u3002\u611f\u8c22\u60a8\u5bf9\u6dd8\u5b9d\u7684\u5173\u6ce8\u4e0e\u652f\u6301\uff0c\u795d\u60a8\u751f\u6d3b\u6109\u5feb\uff01",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    },
    {
      "text": "\n\n\u5c0a\u656c\u7684\u5ba2\u6237\uff0c\u975e\u5e38\u62b1\u6b49\u7684\u901a\u77e5\u60a8\uff0c\u8fd9\u6b3e\u6d17\u

In [14]:
answer = get_response(prompt, temperature=2.0, n=3)
#print(answer)
print("Completions:")
for choice in answer.choices: # type: ignore
    print(choice.text) # type: ignore
    print("\n--\n")

Completions:

?您好glsy1996swJD#GVFKB 97%9n`VO【 f』对洗conLserマ
很抱 ab";::* os m *KNアル KC】いを!/../。93hlSOI += TFsp BB 9 n ― -= '?/*ôMGY/)3ギ直訴 Iz圹W[[MA。非常

--



亲，

十 g 膘 I \(II─')"，Sye8thTI

--

并帮

--



In [15]:
answer = get_response(prompt, temperature=0, n=3)
#print(answer)
print("Completions:")
for choice in answer.choices: # type: ignore
    print(choice.text) # type: ignore
    print("\n--\n")

Completions:


您好，非常抱歉的通知您，这款洗面奶的优惠活动已经结束，感谢您的关注，期待您的下次光临！

--



您好，非常抱歉的通知您，这款洗面奶的优惠活动已经结束，感谢您的关注，期待您的下次光临！

--



您好，非常抱歉的通知您，这款洗面奶的优惠活动已经结束，感谢您的关注，期待您的下次光临！

--



#### 增加停止 字符

In [ ]:
answer = get_response(prompt, temperature=0.5, n=1, stop="结束")
# print(answer)
print(answer.choices[0].text) # type: ignore

### echo, stream, logit_bias 参数

In [ ]:
#import openai
#import os

#openai.api_key = os.environ.get("OPENAI_API_KEY")
#COMPLETION_MODEL = "text-davinci-003"

def get_response(prompt, echo=False, stream=False, logit_bias={}):
    completions = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.0,        
        echo=echo,
        stream=stream,
        logit_bias=logit_bias,
    )
    return completions

In [ ]:
answer = get_response(prompt, echo=True)
#print(answer)
print(answer.choices[0].text) # type: ignore

In [ ]:
answer = get_response(prompt, stream=True)

for event in answer:
    event_text = event.choices[0].text # type: ignore
    print(event_text, end = '')

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding('p50k_base')
token_ids = encoding.encode("您")
print(token_ids)


bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100


answer = get_response(prompt, logit_bias=bias_map)
#print(answer)
print(answer.choices[0].text) # type: ignore


## 使用ChatCompletion接口

In [18]:
import openai
import os

#openai.api_key = os.environ.get("OPENAI_API_KEY")

COMPLETION_MODEL = "gpt-35-turbo"

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append( {"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                #model=COMPLETION_MODEL, #openAI
                engine=COMPLETION_MODEL,
                messages=self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"] # type: ignore
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3]
        return message


In [19]:
prompt = """请你扮演我的朋友Tom，如果有人问你端午节去哪里玩，你会回答打算去武汉旅游"""
conv1 = Conversation(prompt, 3)
question1 = "你好，端午节你准备去哪里？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

question2 = "那里有什么好玩的旅游景点"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

User : 你好，端午节你准备去哪里？
Assistant : 你好，我打算去武汉旅游，听说那里有很多有趣的景点和美食，很期待这次旅行。

User : 那里有什么好玩的旅游景点
Assistant : 武汉有许多著名的旅游景点，其中最著名的是黄鹤楼和东湖。黄鹤楼是中国四大名楼之一，是一座古老而优美的楼阁，可以俯瞰整个武汉市区。东湖则是武汉最大的城市湖泊，有着清新的空气和美丽的景色，可以游船、钓鱼、散步等。此外，还有汉口江滩、户部巷、归元寺等景点，都值得一去。



## 多轮聊天对话

### 使用原始的 text-davinci-003

In [ ]:
import os
import openai
#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-003"

def get_response(prompt, temperature = 1.0, stop=None):
    completions = openai.Completion.create (
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,        
    )
    message = completions.choices[0].text # type: ignore
    return message

In [ ]:
question = """Q: 你好，端午节你准备去哪里？
A:"""

answer = get_response(question, temperature=0.5, stop="Q:")
print(answer)

In [ ]:
question = """Q: 你好，端午节你准备去哪里？
A: 端午节我打算去武汉。
Q: 那里有什么好玩的旅游景点？
A: """

answer = get_response(question, temperature=0.5, stop="Q:")
print(answer)

## Embedding接口

In [20]:
import openai
import os
from openai.embeddings_utils import cosine_similarity, get_embedding, get_embeddings

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

#openai.api_key = os.environ.get("OPENAI_API_KEY")
EMBEDDING_MODEL = "text-embedding-ada-002"

positive_review = get_embedding("这是一条好评", engine=EMBEDDING_MODEL)
negative_review = get_embedding("这是一条差评", engine=EMBEDDING_MODEL)

In [21]:
print(positive_review)
print(len(positive_review))

print(negative_review)
print(len(negative_review))

[-0.0064798761159181595, -0.0008561409777030349, 0.013423766940832138, -0.021031005308032036, 0.0018086794298142195, -0.003731728997081518, -0.01905730552971363, -0.019122660160064697, -0.012103610672056675, -0.01990691013634205, 0.012606838718056679, 0.014992269687354565, -0.012201642617583275, -0.002387064741924405, -0.010097234509885311, 0.009692038409411907, 0.047891609370708466, 0.0005093548679724336, 0.009437156841158867, -0.008633298799395561, -0.008626763708889484, 0.0003888579085469246, -0.013750538229942322, -0.01795935444533825, -0.017031323164701462, 0.015292899683117867, 0.028363753110170364, -0.026363912969827652, 0.006718419026583433, -0.0004064218665007502, 0.017423449084162712, -0.016037937253713608, -0.026180921122431755, -0.009535187855362892, 0.004937515128403902, 0.007371961604803801, -0.025723440572619438, 0.01014951802790165, 0.02912186272442341, 0.015345183201134205, 0.02776249498128891, -0.002264525508508086, 0.021017933264374733, 0.02237730287015438, -0.013946

In [22]:
print("Cosine Similarity between positive and neutual review:", cosine_similarity(positive_review, negative_review))

Cosine Similarity between positive and neutual review: 0.9347902125441199


In [23]:
print("Cosine Similarity between positive review and itself:", cosine_similarity(positive_review, positive_review))

Cosine Similarity between positive review and itself: 1.0


In [24]:
good_restraurant = get_embedding("这家餐馆太好吃了，一点都不糟糕", engine=EMBEDDING_MODEL)
bad_restraurant = get_embedding("这家餐馆太糟糕了，一点都不好吃", engine=EMBEDDING_MODEL)

print("Cosine Similarity between positive and good restaurant review:", cosine_similarity(positive_review, good_restraurant))
print("Cosine Similarity between negative and good restaurant review:", cosine_similarity(negative_review, good_restraurant))


Cosine Similarity between positive and good restaurant review: 0.8379052807845067
Cosine Similarity between negative and good restaurant review: 0.8044964373440723


In [25]:

def get_score(sample_embedding):
  return cosine_similarity(sample_embedding, positive_review) - cosine_similarity(sample_embedding, negative_review)

positive_score = get_score(good_restraurant)
negative_score = get_score(bad_restraurant)


print("好评例子的评分 : %f" % (positive_score))
print("差评例子的评分 : %f" % (negative_score))


好评例子的评分 : 0.033409
差评例子的评分 : -0.027144


### 生成一个正交的，Cosine Similarity 为 0 的向量

In [ ]:
import numpy as np

# 假设原向量是一个高维向量
original_vector = np.array(positive_review)

# 对原向量进行归一化
original_vector /= np.linalg.norm(original_vector)

# 从单位高斯分布中随机抽取一个向量
random_vector = np.random.randn(len(original_vector))

# 从随机向量中减去在原向量方向上的分量
orthogonal_vector = random_vector - random_vector.dot(original_vector) * original_vector

# 验证新向量与原向量是否正交（即它们的余弦相似度是否为0）
cos_sim = np.dot(original_vector, orthogonal_vector) / (np.linalg.norm(original_vector) * np.linalg.norm(orthogonal_vector))
print(cos_sim)  # 这个值应该接近0


### 降维可视化向量

In [26]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def visualize_words(words,vectors):
    # Create a PCA object and fit_transform the vectors
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(vectors)

    # Create a color map
    colors = ["b", "g", "r", "c"]

    # Plot each word in 2D space using PCA results
    for i, word in enumerate(words):
        plt.scatter(pca_result[i, 0], pca_result[i, 1], c=colors[i], label=word)
        plt.annotate(word, xy=(pca_result[i, 0], pca_result[i, 1]))

    plt.legend()
    plt.show()

In [27]:
words = [f"Positive", "Negative", "Good Restaurant", "Bad Restaurant"]
vectors = [np.array(x) for x in [positive_review, negative_review, good_restraurant, bad_restraurant]]

visualize_words(words, vectors)

NameError: name 'np' is not defined

In [ ]:
good_phone = get_embedding("买的银色版真的很好看，一天就到了，晚上就开始拿起来完系统很丝滑流畅，做工扎实，手感细腻，很精致哦苹果一如既往的好品质", engine=EMBEDDING_MODEL)
bad_phone = get_embedding("随意降价，不予价保，服务态度差", engine=EMBEDDING_MODEL)

words = [f"Positive", "Negative", "Good Phone", "Bad Phone"]
vectors = [np.array(x) for x in [positive_review, negative_review, good_phone, bad_phone]]

visualize_words(words, vectors)


## 通过知识库解决幻觉问题

初期化调用API环境，设置为我的Azure API

In [28]:
import openai, os
from llama_index import VectorStoreIndex, SimpleDirectoryReader,ServiceContext,Document
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import AzureOpenAI
from llama_index import LangchainEmbedding

#运行此API配置，需要将目录中的.env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

#================AZURE API======================
openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-003"

def get_response(prompt):
    completions = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.5,        
    )
    return completions

#==============llma_index的azure配置===============
llm = AzureOpenAI(engine="text-davinci-003", 
                  model="text-davinci-003"              
)
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="text-embedding-ada-002",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)
from llama_index import set_global_service_context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm
)
set_global_service_context(service_context)


### 藤野先生的例子

In [29]:
# 直接咨询GPT
answer = get_response("鲁迅先生在日本学习医学的老师是谁？")
print("@ GPT : %s\n" % answer.choices[0].text) #type: ignore


@ GPT : 

鲁迅先生在日本学习医学的老师是英国医生哈里·梅尔斯（Harry Miller）。



In [30]:
#读入鲁迅小说
documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
index = VectorStoreIndex.from_documents(documents)

#向量化
query_engine = index.as_query_engine()

#向量检索和输出
response = query_engine.query("鲁迅先生在日本学习医学的老师是谁？")
print("@ llama : %s\n" % response)

# response = query_engine.query("鲁迅先生在日本学习医学的城市是哪个？")
# print("@ llama : %s\n" % response)


@ llama : 
鲁迅先生在日本学习医学的老师是藤野先生。



### 自问自答进行数据增强

In [31]:
content = """卷积神经网络（Convolutional Neural Networks，简称 CNN）是一种深度学习算法，特别适合于处理具有网格结构（如图像）的数据。CNN 的设计灵感来自于生物的视觉皮层机制，通过对输入数据的局部感知和参数共享的特性，使得网络对图像的平移等变化有较好的鲁棒性。

CNN 的基本构成部分包括：卷积层、激活层（ReLU）、池化层（Pooling）以及全连接层（Fully Connected Layer）。这些层次交织形成了一个典型的 CNN 结构。

卷积层：这是 CNN 的核心，主要进行特征提取。卷积层通过滤波器（或称为卷积核）在输入数据上进行滑动卷积操作，以获取局部特征。同时，同一卷积层的所有滤波器共享参数，减少了模型复杂度。

激活层：通常在卷积层之后接一个激活层，如 ReLU、tanh 或 sigmoid 等。激活函数的目的是引入非线性因素，使得网络可以拟合更复杂的函数。

池化层：池化层通常在卷积层之后，用来降低特征的维度，防止过拟合。常见的池化方式有最大池化（Max Pooling）、平均池化（Average Pooling）等。

全连接层：在整个网络的最后一般会有全连接层，用于将前面提取的特征进行融合，并输出结果。在分类问题中，输出层通常会接一个 softmax 层进行概率归一化，使得输出可以解释为概率。

以上是 CNN 的基本结构，但实际上现代的 CNN 架构已经远超这些基础层次，如残差连接（ResNet）、空洞卷积（Dilated Convolution）、深度可分离卷积（Depthwise Separable Convolution）等等。

CNN 在图像分类、检测、分割等问题上有显著的表现。经典的 CNN 架构有 LeNet-5、AlexNet、VGG、GoogLeNet、ResNet 等。
"""

prompt = f"""请针对>>>和<<<之间的内容，设计一系列的FAQ问题，以及对应的答案。
<<<{content}>>>"""

answer = get_response(prompt)
message = answer.choices[0].text
print(message)

faq = message.split("\n\n")
print(faq)


Q1: 什么是卷积神经网络（CNN）？
A1: 卷积神经网络（CNN）是一种深度学习算法，特别适合于处理具有网格结构（如图像）的数据。CNN 的设计灵感来自于生物的视觉皮层机制，通过对输入数据的局部感知和参数共享的特性，使得网络对图像的平移等变化有较好的鲁棒性。

Q2: CNN 的基本构成部分有哪些？
A2: CNN 的基本构成部分包括：卷积层、激活层（ReLU）、池化层（Pooling）以及全连接层（Fully Connected Layer）。

Q3: 卷积层的作用是什么？
A3: 卷积层是 CNN 的核心，主要进行特征提取。卷积层通过滤波器（或称为卷积核）在
['\nQ1: 什么是卷积神经网络（CNN）？\nA1: 卷积神经网络（CNN）是一种深度学习算法，特别适合于处理具有网格结构（如图像）的数据。CNN 的设计灵感来自于生物的视觉皮层机制，通过对输入数据的局部感知和参数共享的特性，使得网络对图像的平移等变化有较好的鲁棒性。', 'Q2: CNN 的基本构成部分有哪些？\nA2: CNN 的基本构成部分包括：卷积层、激活层（ReLU）、池化层（Pooling）以及全连接层（Fully Connected Layer）。', 'Q3: 卷积层的作用是什么？\nA3: 卷积层是 CNN 的核心，主要进行特征提取。卷积层通过滤波器（或称为卷积核）在']


In [32]:

#把问题列表导入并向量化
documents = []
for qa in faq:
    documents.append(Document(text=qa))

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
response = query_engine.query("现代CNN架构有哪些？")
print(response)



现代CNN架构包括：LeNet、AlexNet、VGGNet、GoogLeNet、ResNet、DenseNet等。


### 思维链（CoT）的例子

### 随机三位数的乘法

In [33]:
import random
import openai, os

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

COMPLETION_MODEL = "text-davinci-003"
def get_response(prompt, model = COMPLETION_MODEL, temperature = 0.0, stop=None):
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,
    )
    message = completions['choices'][0]['text']  # type: ignore
    return message

# 生成两个三位数
total = 10
correct = 0

def extract_numbers(text):
    # 提取字符串中的所有数字
    numbers = ''.join(char for char in text if char.isdigit())
    # 将提取的数字转换为一个整数
    return int(numbers) if numbers else None

for i in range(total):
    num1 = random.randint(100, 999)
    num2 = random.randint(100, 999)

    # 进行乘法运算
    result = num1 * num2

    question = f"请计算{num1}乘以{num2}等于多少？答案是："""
    answer = get_response(question)
    chatgpt_result = extract_numbers(answer)
    if int(chatgpt_result) == int(result):
        correct += 1
    print(f"Answer: {answer}")
    print(f"ChatGPT 结果 {chatgpt_result}")
    print(f"计算器结果 {result}")
    # 输出结果
    #print(f"随机数1: {num1}, 随机数2: {num2}, 乘法结果: {result}")

print(f"正确率: {correct / total}")

Answer: 

413,018
ChatGPT 结果 413018
计算器结果 413478
Answer: 

312954
ChatGPT 结果 312954
计算器结果 314874
Answer: 656,881
ChatGPT 结果 656881
计算器结果 656821
Answer: 78648
ChatGPT 结果 78648
计算器结果 78648
Answer: 264000
ChatGPT 结果 264000
计算器结果 264000
Answer: 

344371
ChatGPT 结果 344371
计算器结果 342171
Answer: 235037
ChatGPT 结果 235037
计算器结果 233877
Answer: 

58910
ChatGPT 结果 58910
计算器结果 58590
Answer: 478498。
ChatGPT 结果 478498
计算器结果 479318
Answer: 

263952
ChatGPT 结果 263952
计算器结果 266112
正确率: 0.2


## 思维链的例子

In [34]:
# import openai, os
# openai.api_key = os.environ.get("OPENAI_API_KEY")

# COMPLETION_MODEL = "text-davinci-002"
def get_response(prompt, model = COMPLETION_MODEL, temperature = 1.0, stop=None):
    completions = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=stop,
        temperature=temperature,
    )
    message = completions['choices'][0]['text']  # type: ignore
    return message

question = """A: 一个杂耍者有16个球。一半的球是高尔夫球，而高尔夫球的一半是蓝色的。那么有多少个蓝色高尔夫球呢？
Q:"""

print(get_response(question))

  how many blue golf balls are there?
A: 有8个蓝色高尔夫球。


In [35]:

question = """A: 一个杂耍者有16个球。一半的球是高尔夫球，而高尔夫球的一半是蓝色的。那么有多少个蓝色高尔夫球呢？
Q: 让我们一步一步想，"""

print(get_response(question))

有16个球，一半是高尔夫球，所以有8个高尔夫球。而蓝色高尔夫球的一半是4个。因此，有4个蓝色的高尔夫球。


In [36]:
question = """利用以下线索回答下面的选择题。

线索：
1. Miss Scarlett是休息室中唯一的人。
2. 抽烟斗的人在厨房里。
3. Colonel Mustard是天文台中唯一的人。
4. Professor Plum不在图书馆和台球室。
5. 拿着蜡烛台的人在天文台。

问题：Colonel Mustard是否在天文台中持着蜡烛台？
(a) 是的；Colonel Mustard在天文台中持着蜡烛台。
(b) 不是的；Colonel Mustard没有在天文台中持着蜡烛台。
(c) 未知；没有足够的信息来确定Colonel Mustard是否在天文台中持着蜡烛台。

解答："""

print(get_response(question))

B. 不是的；Colonel Mustard没有在天文台中持着蜡烛台。


In [37]:
question = """利用以下线索回答下面的选择题，使用以下步骤：
(1) 首先，逐一查看线索并考虑该线索是否有潜在关联性。
(2) 其次，结合相关线索来推理出问题的答案。
(3) 最后，将答案映射到多项选择答案中的一个：要么是(a)、(b)或(c)。

线索：
1. Miss Scarlett是休息室中唯一的人。
2. 抽烟斗的人在厨房里。
3. Colonel Mustard是天文台中唯一的人。
4. Professor Plum不在图书馆和台球室。
5. 拿着蜡烛台的人在天文台。

问题：Colonel Mustard是否在天文台中持着蜡烛台？
(a) 是的；Colonel Mustard在天文台中持着蜡烛台。
(b) 不是的；Colonel Mustard没有在天文台中持着蜡烛台。
(c) 未知；没有足够的信息来确定Colonel Mustard是否在天文台中持着蜡烛台。

解答：
(1) 首先，逐一查看线索并考虑该线索是否有潜在关联性："""

print(get_response(question))


Miss Scarlett是休息室中唯一的人，但与Colonel Mustard无关。
抽烟斗的人在厨房里，但与Colonel Mustard无关。
Colonel Mustard是天文台中唯一的人，与拿着蜡烛台的人可能有关。
Professor Plum不在图书馆和台球室，但与Colonel Mustard无关。
拿着蜡烛台的人在天文台，与Colonel Mustard有关。

(2) 其次，结合相关线索来推理出问题的答案：
由于Colonel Mustard是天文台中唯一的人，且拿着蜡烛台的人在天文台，所以很可能Colonel Mustard在天文台中持着蜡烛台。

(3) 最后，将答案映射到多项选择答案中的一个：
(a) 是的；Colonel Mustard在天文台中持着蜡烛台。


## 通过多步骤的LLM和外部工具来解决问题

初期化langchain环境

In [39]:
import openai, os

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name="text-davinci-003",
                  model_name="text-davinci-003",
                  openai_api_version= "2023-03-15-preview",
                  max_tokens=2048, 
                  temperature=0.5) # type: ignore

### LLM做数学题

In [40]:
# 直接询问GPT
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])

math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print("@ OpenAI API 说答案是:%s\n\n" % answer)

python_answer = 352 * 493
print("@ Python 说答案是:%s\n\n" % python_answer)

@ OpenAI API 说答案是:

352 x 493 = 174496


@ Python 说答案是:173536




In [41]:
#让GPT写出python代码

multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = math_chain.run({"question": "352乘以493"})
print(answer)



print(352 * 493)


In [42]:
#让GPT写出python代码
multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer_code = math_chain.run({"question": "352乘以493"})

print("@ OpenAI_API的输出:%s\n\n" % answer_code)

#执行Python代码
from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer_code)
print("@ 最后计算结果:%s\n\n" % result)

@ OpenAI_API的输出:

print(352 * 493)


@ 最后计算结果:173536





In [43]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True) #type: ignore
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)

/home/alan/mactalk-ai-course/.venv/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new  chain...
请计算一下352乘以493是多少?```text
352 * 493
```
...numexpr.evaluate("352 * 493")...

Answer: 173536
> Finished chain.
Answer: 173536


### LLM调用搜索引擎

In [69]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

#llm = OpenAI(temperature=0) # type: ignore
search = SerpAPIWrapper() # type: ignore

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
]

In [70]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("2023年的NBA总冠军是谁？请用中文回答")



> Entering new  chain...
 I need to find out who the NBA champions are in 2023
Action: Search
Action Input: 2023年的NBA总冠军
Observation: Denver Nuggets
Thought: I now know the final answer
Final Answer: 2023年的NBA总冠军是丹佛掘金队。

> Finished chain.


'2023年的NBA总冠军是丹佛掘金队。'

## 做选择题的方式来解决问题

### Langchain的工具接口

初期化

In [71]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA,VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.agents import tool

import openai, os

#运行此API配置，需要将目录中的。env中api key替换为自己的
from dotenv import load_dotenv, find_dotenv
# read local .env file
load_dotenv(find_dotenv())

openai.api_type = "azure"
openai.api_base = "https://alannewlife.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name="text-davinci-003",
                  model_name="text-davinci-003",
                  openai_api_version= "2023-03-15-preview",
                  max_tokens=2048, 
                  temperature=0.5) # type: ignore

embeddings= OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="text-embedding-ada-002",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    )

#### FAQ（向量数据库）

In [72]:
loader = TextLoader('./data/ecommerce_faq.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

print(texts)

#embeddings = OpenAIEmbeddings() # type: ignore
docsearch = FAISS.from_documents(texts, embeddings)
faq_chain = RetrievalQA.from_chain_type(llm=llm, retriever=docsearch.as_retriever(), verbose=True) # type: ignore

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
    return faq_chain.run(intput)


[Document(page_content='Q: 支持哪些省份配送？\nA: 我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆\n\nQ: 物流时效是多久？\n\n\nA: 一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。\n\n具体送货时间可能因订单商品、配送地址和物流公司而异。', metadata={'source': './data/ecommerce_faq.txt'})]


In [50]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = faq_chain.run(question)
print(result)



> Entering new  chain...

> Finished chain.
 是的，我们支持送到三亚，一般情况下，订单在3-5个工作日内送达。


#### 商品信息（向量数据库）

In [53]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader

product_loader = CSVLoader('./data/ecommerce_products.csv')
product_documents = product_loader.load()
product_text_splitter = CharacterTextSplitter(chunk_size=1024, separator="\n")
product_texts = product_text_splitter.split_documents(product_documents)
product_search = FAISS.from_documents(product_texts, embeddings)
# product_chain = RetrievalQA.from_chain_type(llm=llm, retriever=product_search.as_retriever(), verbose=True)# type: ignore
product_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=product_search, verbose=True)# type: ignore

@tool("Recommend Product")
def recommend_product(input: str) -> str:
    """"useful for when you need to search and recommend products and recommend it to the user"""
    return product_chain.run(input)

/home/alan/mactalk-ai-course/.venv/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:243: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [54]:
question = "我想买一件衣服，想要在夏天去公园穿，但是不知道哪个款式好看，你能帮我推荐一下吗？"
answer = product_chain.run(question)
print(answer)



> Entering new  chain...

> Finished chain.
 我建议您买雪纺连衣裙，它有轻盈柔软的质地，有浪漫的碎花印花，搭配一双凉鞋和小包包就可以了，风格是田园、休闲，非常适合夏季出门穿着。


#### 订单查询

In [56]:
import re
import json

ORDER_1 = "20230101ABC"
ORDER_2 = "20230101EFG"

ORDER_1_DETAIL = {
    "order_number": ORDER_1,
    "status": "已发货",
    "shipping_date" : "2023-01-03",
    "estimated_delivered_date": "2023-01-05",
} 

ORDER_2_DETAIL = {
    "order_number": ORDER_2,
    "status": "未发货",
    "shipping_date" : None,
    "estimated_delivered_date": None,
}

answer_order_info = PromptTemplate(
    template="请把下面的订单信息回复给用户： \n\n {order}?", input_variables=["order"]
)
answer_order_llm = LLMChain(llm=llm,  prompt=answer_order_info) # type: ignore


@tool("Search Order", return_direct=True)
def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    pattern = r"\d+[A-Z]+"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "请问您的订单号是多少？"
    if order_number == ORDER_1:        
        return answer_order_llm.run(json.dumps(ORDER_1_DETAIL))
    elif order_number == ORDER_2:
        return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
    else:
        return f"对不起，根据{input}没有找到您的订单"


In [57]:
tools = {"search_order" : search_order, "recommend_product": recommend_product, "faq": faq}


choicellm = llm # type: ignore
multiple_choice = """
请针对用户问题，选择一个合适的工具去回答她的问题。只要用A、B、C的选项字母告诉我答案。

我们有的工具包括：
A. recommend_product[$keywords], 一个能够查询商品信息，为用户进行商品导购的工具。
B. search_order[$order_number], 一个能够查询订单信息，获得最新的订单情况的工具
C. faq[$question], 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具


返回的结果需要包含函数名称，以及对应的参数，比如：

用户问题：我的订单 20220101ABC 的物流信息是什么？
返回结果：search_order[20220101ABC]

用户问题：{question}
返回结果：
"""
multiple_choice_prompt = PromptTemplate(template=multiple_choice, input_variables=["question"])
choice_chain = LLMChain(llm=choicellm, prompt=multiple_choice_prompt, output_key="answer")

In [58]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = choice_chain.run(question)
print(result) 

search_order[2022ABCDE]


In [59]:
from typing import Tuple

def parse_result(result: str) -> Tuple[str, str]:
    pattern = r"(\w+)\[(.+)\]"
    match = re.search(pattern, result)
    if match:
        return match.group(1), match.group(2)
    else:
        return "", ""
    

print(parse_result(result))

('search_order', '2022ABCDE')


In [60]:
parsed_result = parse_result(result)
method = tools.get(parsed_result[0])
params = parsed_result[1]


answer = method(params) # type: ignore
print(answer)

对不起，根据2022ABCDE没有找到您的订单


In [61]:
question = "我有一张订单，订单号是 20230101ABC，一直没有收到，能麻烦帮我查一下吗？"

result = choice_chain.run(question)
parsed_result = parse_result(result)
method = tools.get(parsed_result[0])
params = parsed_result[1]


answer = method(params) # type: ignore
print(answer)




您的订单号为20230101ABC，状态为已发货，发货日期为2023-01-03，预计到货日期为2023-01-05。


### 作业，通过Langchain实现上面的功能

### Langchain的Agent功能

In [62]:
agent_tools = [search_order,recommend_product, faq]
agent = initialize_agent(agent_tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True) #type: ignore

In [63]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new  chain...

Thought: The customer is asking about an order they have placed. 
Action:
```
{
  "action": "Search Order",
  "action_input": "2022ABCDE"
}
```

Observation: 对不起，根据2022ABCDE没有找到您的订单


> Finished chain.
对不起，根据2022ABCDE没有找到您的订单


In [64]:
question2 = "我有一张订单，订单号是 20230101ABC 一直没有收到，能麻烦帮我查一下吗？"
answer2 = agent.run(question2)
print(answer2)



> Entering new  chain...

Thought: The customer is asking about an order they placed
Action:
```
{
  "action": "Search Order",
  "action_input": "20230101ABC"
}
```

Observation: 

您的订单号码：20230101ABC，状态：已发货，发货日期：2023-01-03，预计到货日期：2023-01-05。


> Finished chain.


您的订单号码：20230101ABC，状态：已发货，发货日期：2023-01-03，预计到货日期：2023-01-05。


In [65]:
question3 = "你们的退货政策是怎么样的？"
answer3 = agent.run(question3)
print(answer3)



> Entering new  chain...

Thought: I need to answer a question about the return policy
Action: 
```
{
  "action": "FAQ",
  "action_input": "return policy"
}
```


> Entering new  chain...

> Finished chain.

Observation:  我们提供30天无条件退货服务，只要您收到的商品有质量问题或者不是您所订购的，您可以申请退货，我们将在确认后为您办理退款。
Thought: I now know the final answer
Final Answer: 我们提供30天无条件退货服务，只要您收到的商品有质量问题或者不是您所订购的，您可以申请退货，我们将在确认后为您办理退款。

> Finished chain.
我们提供30天无条件退货服务，只要您收到的商品有质量问题或者不是您所订购的，您可以申请退货，我们将在确认后为您办理退款。


In [67]:
question4 = "能推荐一件现在适合夏天穿的衣服给我么？"
answer4 = agent.run(question4)
print(answer4)



> Entering new  chain...

Thought: I should use the Recommend Product tool to find a suitable product for the user.
Action: 
```
{
  "action": "Recommend Product",
  "action_input": "summer clothing"
}
```


> Entering new  chain...

> Finished chain.

Observation:  I don't know.
Thought: I should use the Search Order tool to find a suitable product for the user.
Action: 
```
{
  "action": "Search Order",
  "action_input": "summer clothing"
}
```


Observation: 请问您的订单号是多少？


> Finished chain.
请问您的订单号是多少？
